In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm_notebook as tqdm

import torchvision
# Then package torchvision, that has data loaders for common datasets such as 
# Imagenet, CIFAR10, MNIST, etc. and data transformers for images, viz., 
# torchvision.datasets and torch.utils.data.DataLoader.
import torchvision.transforms as transforms

# CIFAR10 dataset
#     classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. 
#     images: size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.
# The output of torchvision datasets are PILImage images of range [0, 1], which then we have to transform 
# into normalized Tensors with range [-1, 1]

from tensorboardX import SummaryWriter

%matplotlib inline
random.seed(31)
torch.manual_seed(31)


# Utility functions

In [2]:
def image_show(img):
    img = img / 2 + 0.5 #normalization
    np_image = img.numpy()
    plt.figure(figsize=(10,10))
    plt.imshow(np.transpose(np_image, (1, 2, 0)))
    plt.show()

def image_grid_show(image_grid):
    image_show(torchvision.utils.make_grid(image_grid, nrow=batch_size//2))

# Network class definition


In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 3 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operations: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) # prepare the input for the fully connected layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)       

# Train the network

## Default parameters

In [13]:
batch_size=10
epochs=3
learning_rate=0.001
momentum=0.9
log_event_path="./logs"

## Data preparation & exploration

In [14]:
# Load and normalizing the CIFAR10 training and test datasets using torchvision
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Network instantiation and start the training loop

In [18]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [16]:
# Setup tensorboard writers
from datetime import datetime
suffix = f"run-{datetime.now()}".replace(" ", "_").replace(":", "_")
writer = SummaryWriter("/".join([log_event_path, suffix]))


In [17]:
for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = 0.0
    for iteration, data in tqdm(enumerate(train_loader, 0), desc= "Batches"):
        inputs, labels = data
        # clean up all the parameter gradients
        optimizer.zero_grad()
        
        print(len(inputs))
        break
        
        # forward + backward + optimize 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()            
        optimizer.step()
       
        # print statistics
        running_loss += loss.item()   # `.item()` to get the value of a tensor
        if iteration % 2000 == 1999:
            print(f"Epoch {epoch + 1}, iteration {iteration + 1} -> loss: {(running_loss / 2000):.3f}")
            running_loss = 0.0
        
        # tensorboardX event logs
        writer.add_scalar('iter loss', running_loss, iteration)
    
    avg_loss = running_loss / (len(train_loader))
    writer.add_scalar('epoch loss', avg_loss, epoch)

print("Training finished")

10



10



10


Training finished


# Evaluate the model

In [31]:
# classify the full test set
correct = 0
total = 0
with torch.no_grad():                      # avoid keeping the gradients
    for data in test_loader:
        inputs, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on {total} test images: {(100 * correct / total):.2f}")

Accuracy on 10000 test images: 10.04


In [9]:
# Measuring acc per class
class_correct = [0] * len(classes)
class_total = [0] * len(classes)
with torch.no_grad():
    for i, data in tqdm(enumerate(test_loader), total=10000//batch_size, desc="Batches"):
        inputs, labels = data
        outputs = net(images)
        # the net returns a tensor per image 
        # containing the classification 'score' per classs.
        # Thus, we use `.max` to identify the most probable classification
        _, predicted = torch.max(outputs, 1)
        is_correct = (predicted == labels).squeeze()
        for i in range(len(labels)):
            true_label = labels[i]
            class_correct[true_label] += is_correct[i].item()
            class_total[true_label] += 1

for i in range(len(classes)):
    print(f"Accuracy of {classes[i]:5s} : {(100 * class_correct[i] / class_total[i]):.2f}%")
            


Accuracy of plane : 0.00%
Accuracy of car   : 0.00%
Accuracy of bird  : 20.90%
Accuracy of cat   : 20.10%
Accuracy of deer  : 9.30%
Accuracy of dog   : 0.00%
Accuracy of frog  : 21.40%
Accuracy of horse : 19.60%
Accuracy of ship  : 10.60%
Accuracy of truck : 0.00%
